## Notebook for spike sorting from .rhd data using the Pipeline of mountainsort (linux channel has pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from importlib import reload

# mountainlab imports
from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)

logger.info('Logger set')

/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
2018-11-28 15:31:44,062 - root - INFO - Logger set


In [2]:
os.environ['OMP_NUM_THREADS'] = '8'
os.environ['MKL_NUM_THREADS'] = '8'
os.environ['NUMEXPR_NUM_THREADS'] = '8'

### Session parameters and raw files

In [3]:
exp_base = os.path.abspath('/home/zinch/data/')
exp_mnt = os.path.abspath('/mnt/zuperfinchjr/Data/')

file_sys = {'exp_base': exp_base}

sess_par = {'bird': 'o8g8',
           'sess': 'chronic_20181126_3325_01',
            'neural': 'neural',
           'rec': 0}

file_sys_defaults = {'exp_base': os.path.abspath('/data/experiment')}


def file_names(sess_par, file_sys=None):
    f_s = unroll_file_sys(file_sys_defaults, file_sys)    
    
    f_n = {}
    f_n['raw'] = os.path.join(f_s['raw'], bird)

In [10]:
exp_base

'/home/zinch/data'

In [4]:
raw_folder = os.path.join(exp_mnt, sess_par['bird'], sess_par['neural'], sess_par['sess'])
logger.info('Raw folder {}'.format(raw_folder))
glob.glob(os.path.join(raw_folder, '*.rhd'))

2018-11-28 15:31:51,899 - root - INFO - Raw folder /mnt/zuperfinchjr/Data/o8g8/neural/chronic_20181126_3325_01


['/mnt/zuperfinchjr/Data/o8g8/neural/chronic_20181126_3325_01/raw_intan_181126_162415.rhd',
 '/mnt/zuperfinchjr/Data/o8g8/neural/chronic_20181126_3325_01/raw_intan_181126_163415.rhd',
 '/mnt/zuperfinchjr/Data/o8g8/neural/chronic_20181126_3325_01/raw_intan_181126_164415.rhd']

In [5]:
rhd_files = glob.glob(os.path.join(raw_folder, '*.rhd'))
rhd_files.sort()
pd_f = pd.DataFrame(rhd_files, columns=['f_path'])

pd_f['stamp'] = pd_f['f_path'].apply(lambda x: int(os.path.split(x)[-1].split('.')[0].split('_')[-1]))

# Select some of the files
sess_start = 0
sess_end = 235959
pd_session = pd_f.loc[(pd_f['stamp'] > sess_start) & (pd_f['stamp'] < sess_end), :]
pd_session.index.size

rhd_list = pd_session['f_path'].values.tolist()[:]

## Scripts for sorting

### prep the files with their nice formats, locations and names


In [6]:
rhd_list

['/mnt/zuperfinchjr/Data/o8g8/neural/chronic_20181126_3325_01/raw_intan_181126_162415.rhd',
 '/mnt/zuperfinchjr/Data/o8g8/neural/chronic_20181126_3325_01/raw_intan_181126_163415.rhd',
 '/mnt/zuperfinchjr/Data/o8g8/neural/chronic_20181126_3325_01/raw_intan_181126_164415.rhd']

In [7]:
ss_folder = os.path.join(exp_base, sess_par['bird'], 'ss_data', sess_par['sess'])

file_paths, out_folder = msc.make_paths(ss_folder)
os.makedirs(ss_folder, exist_ok=True)
#todo:
# meta and pars from intan files et al

s_f = 20000 #todo: this should come from the recording system metadata
ds_params = {'samplerate': s_f,
            'detect_sign': -1}

with open(file_paths['params'], 'w') as fp:
    json.dump(ds_params, fp)
    logger.info('Created session par files {}'.format(file_paths['params']))


2018-11-28 15:32:00,349 - root - INFO - Created session par files /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/params.json


In [8]:
# create the mda file from the rhd_list
intan_meta, samples_in = intan.intan_to_bin(rhd_list[:], file_paths['mda'], variant='mda')

2018-11-28 15:32:02,303 - pipefinch.neural.convert - INFO - Creating the output file /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/raw.mda
2018-11-28 15:32:02,303 - pipefinch.neural.convert - INFO - File 0/3
notch filter: 100%|██████████| 17/17 [00:01<00:00, 11.50it/s]
2018-11-28 15:32:22,465 - pipefinch.neural.convert - INFO - File 1/3
notch filter: 100%|██████████| 17/17 [00:01<00:00, 11.96it/s]
2018-11-28 15:32:42,240 - pipefinch.neural.convert - INFO - File 2/3
notch filter: 100%|██████████| 17/17 [00:01<00:00, 13.83it/s]


In [9]:
file_paths

{'mda': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/raw.mda',
 'params': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/params.json',
 'geom': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/geom.csv',
 'filt': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/filt.mda.prv',
 'pre': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/pre.mda.prv',
 'firings': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/firings.mda',
 'firings_curated': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/firings_curated.mda',
 'cluster_metrics': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/cluster_metrics.json'}

In [12]:
msc.read_dataset_params(ss_folder)

{'samplerate': 20000, 'detect_sign': -1}

In [13]:
reload(msc)

<module 'pipefinch.neural.sort.mountain.core' from '/home/zinch/repos/pipefinch/pipefinch/neural/sort/mountain/core.py'>

In [14]:
from mountainlab_pytools import mlproc as mlp
Pipeline=mlp.initPipeline()

JSProxyWidget(status='Not yet rendered')

In [15]:
output_dir = os.path.join(ss_folder, 'sort_out')
# Pipeline was failing, need to debug this ('NameError: name 'widgets' is not defined')
# with Pipeline:
#     msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1, detect_threshold=3, dispatch_method='add')

msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1, detect_threshold=2, freq_min=600, dispatch_method='run')

2018-11-28 15:36:34,826 - pipefinch.sort.mountain.comre - INFO - Bandpass filter


RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/raw.mda --parameters freq_max:6000 freq_min:600 samplerate:20000 --outputs timeseries_out:/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/filt.mda.prv


2018-11-28 15:37:01,576 - pipefinch.sort.mountain.comre - INFO - Whitening


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: a14840baee91ab39a8d6ae2b2b4549cfdd3271fe
[ Checking outputs... ]
{"timeseries_out":"/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/filt.mda.prv"}
Processing ouput - /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/filt.mda.prv
false
{"timeseries_out":"/tmp/mountainlab-tmp/output_a14840baee91ab39a8d6ae2b2b4549cfdd3271fe_timeseries_out.mda"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /tmp/mountainlab-tmp/output_a14840baee91ab39a8d6ae2b2b4549cfdd3271fe_timeseries_out.mda
false
[ Initializing process ... ]
[ Running ... ] /home/zinch/conda/envs/msort/bin/python3 /home/zinch/conda/envs/msort/etc/mountainlab/packages/ml_ephys/preprocessing/preprocessing.py.mp ephys.bandpass_filter --_tempdir=/tmp/mountainlab-tmp/tempd

2018-11-28 15:37:18,029 - pipefinch.sort.mountain.comre - INFO - Sorting


...

RUNNING: ml-run-process ms4alg.sort --inputs timeseries:/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/pre.mda.prv --parameters adjacency_radius:-1 detect_sign:-1 detect_threshold:2 --outputs firings_out:/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/firings.mda
[ Getting processor spec... ]
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
[ Checking inputs and substituting prvs ... ]
return f(*args, **kwds)
[ Computing process signature ... ]



2018-11-28 15:42:38,618 - pipefinch.sort.mountain.comre - INFO - Getting cluster metrics


Process signature: a956aa64082abc3dbbb36d8f833acc20429a5a0a
[ Checking outputs... ]
{"firings_out":"/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/firings.mda"}
Processing ouput - /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/firings.mda
false
{"firings_out":"/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/firings.mda"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/firings.mda
false
[ Initializing process ... ]
[ Running ... ] /home/zinch/conda/envs/msort/bin/python3 /home/zinch/conda/envs/msort/etc/mountainlab/packages/ml_ms4alg/ms4alg_spec.py.mp ms4alg.sort --_tempdir=/tmp/mountainlab-tmp/tempdir_a956aa6408_3VLbUB --timeseries=/tmp/mountainlab-tmp/tempdir_a956aa6408_3VLbUB/input_timeseries_nAWK3roj.mda --geom= --firings_out=/tmp/mountainlab-tmp/tempdir_a956aa6

2018-11-28 15:43:14,405 - pipefinch.sort.mountain.comre - INFO - Automatically curating


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 083963ce1cf75bd23feaae62ed9dbb4beeebe89d
[ Checking outputs... ]
{"metrics_out":"/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/cluster_metrics.json"}
Processing ouput - /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/cluster_metrics.json
false
{"metrics_out":"/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/cluster_metrics.json"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/cluster_metrics.json
false
[ Initializing process ... ]
[ Running ... ] /home/zinch/conda/envs/msort/etc/mountainlab/packages/ms3.mp ms3.combine_cluster_metrics --_tempdir=/tmp/mountainlab-tmp/tempdir_083963ce1c_of3jwM --metrics_list=/tmp/mountainlab-tmp/tempdir_08

In [16]:
msc.sort_dataset?

Signature: msc.sort_dataset(*, file_paths:dict, freq_min:int=300, freq_max:int=6000, adjacency_radius:int=1, detect_threshold:float=3, dispatch_method:str='run', opts:dict={})
Docstring: <no docstring>
File:      ~/repos/pipefinch/pipefinch/neural/sort/mountain/core.py
Type:      function


In [17]:
file_paths

{'mda': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/raw.mda',
 'params': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/params.json',
 'geom': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/geom.csv',
 'filt': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/filt.mda.prv',
 'pre': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/pre.mda.prv',
 'firings': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/firings.mda',
 'firings_curated': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/firings_curated.mda',
 'cluster_metrics': '/home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/cluster_metrics.json'}

In [18]:
cmd = 'env'
p = subprocess.Popen([cmd], stdout=subprocess.PIPE)
ans = p.communicate()
print(ans)

(b'CLUTTER_IM_MODULE=xim\nCONDA_SHLVL=1\nLS_COLORS=rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.Z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:*.xz=01;31:*.zst=01;31:*.tzst=01;31:*.bz2=01;31:*.bz=01;31:*.tbz=01;31:*.tbz2=01;31:*.tz=01;31:*.deb=01;31:*.rpm=01;31:*.jar=01;31:*.war=01;31:*.ear=01;31:*.sar=01;31:*.rar=01;31:*.alz=01;31:*.ace=01;31:*.zoo=01;31:*.cpio=01;31:*.7z=01;31:*.rz=01;31:*.cab=01;31:*.wim=01;31:*.swm=01;31:*.dwm=01;31:*.esd=01;31:*.jpg=01;35:*.jpeg=01;35:*.mjpg=01;35:*.mjpeg=01;35:*.gif=01;35:*.bmp=01;35:*.pbm=01;35:*.pgm=01;35:*.ppm=01;35:*.tga=01;35:*.xbm=01;35:*.xpm=01;35:*.tif=01;35:*.tiff=01;35:*.png=01;35:*.svg=01;35:*.svgz=01;35:*.mng=01;35:*.pcx=01;3

In [19]:
show_args_dict = {'raw': file_paths['mda'], 
             'filt': file_paths['filt'],
             'pre': file_paths['pre'],
            'samplereate': ds_params['samplerate'],
            'firings': file_paths['firings_curated'],
            'cluster_metrics': file_paths['cluster_metrics']}

cmd_args = ['--{} {}'.format(k, v) for k, v in show_args_dict.items()]
cmd = 'qt-mountainview'

logger.info('View command is {}'.format([cmd, ' '.join(cmd_args)]))
p = subprocess.Popen([cmd, ' '.join(cmd_args)], stdout=subprocess.PIPE)


2018-11-28 15:44:20,906 - root - INFO - View command is ['qt-mountainview', '--raw /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/raw.mda --filt /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/filt.mda.prv --pre /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/pre.mda.prv --samplereate 20000 --firings /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/firings_curated.mda --cluster_metrics /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/cluster_metrics.json']


In [20]:
print(' '.join(['qt-mountainview'] + cmd_args))

qt-mountainview --raw /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/raw.mda --filt /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/filt.mda.prv --pre /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/pre.mda.prv --samplereate 20000 --firings /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/firings_curated.mda --cluster_metrics /home/zinch/data/o8g8/ss_data/chronic_20181126_3325_01/sort_out/cluster_metrics.json


# other viewing examples

ev-templates output/templates.mda.prv
ev-timeseries dataset/raw.mda.prv --firings output/firings.mda.prv --samplerate=30000
qt-mountainview --raw dataset/raw.mda.prv --filt output/filt.mda.prv --pre output/pre.mda.prv --samplerate 30000 --firings output/firings.mda.prv
